# Week 3 - Part 2

## Populate and dataframe from wikipedia (see Part 1 for code explanation etc...)

In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tables=soup.find_all('table', class_='wikitable')
table = tables[0]
table = [[t.get_text().strip() for t in tr.find_all('td')] for tr in table.findAll("tr")][1:]
df = pd.DataFrame(table)
df.columns = ['PostalCode', 'Borough','Neighborhood']
df.drop(df[df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)
df.set_index('PostalCode',inplace=True)
df['Neighborhood']=df.groupby('PostalCode')['Neighborhood'].aggregate(', '.join)
df.reset_index(inplace=True)
df.drop_duplicates(subset='PostalCode',inplace=True)
idx=df.Neighborhood == 'Not assigned'
df.loc[idx, 'Neighborhood'] =df.loc[idx, 'Borough']
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# Load Geo data (not working (limits) see below)

In [8]:
import geocoder # import geocoder

def decode_postal_code(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return (latitude,longitude)

In [9]:
# decode_postal_code('M3A')

# Load Geo data (google not working will use CSV)

In [18]:
geo_url= 'http://cocl.us/Geospatial_data'
geo_df = pd.read_csv(geo_url)
geo_df.columns = ['PostalCode','Latitude','Longitude']
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
merged_df=pd.merge(df, geo_df, on='PostalCode')
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
